# Plotting: thick maps

This notebook documents how to make maps inside the simulation domain with a non-zero thickness.
This is also sometimes called a column density plot, although it generally only applies to gas density,
while any quantity can be used in thick maps.

We use the `map` function to compute a 3D grid of values,
and this volume then gets integrated along the third dimension.

<div class="alert alert-warning">

**Warning**

The current implementation generates a 3D grid of points,
and for each one of these points, finds the single AMR cell that contains it.
It then assigns the value in the AMR cell to the grid point position.
The algorithm does **not** perform a histogramming of all the cells into a 3D cube of bins.

This means that this method is not well suited to the computation of, e.g., surface density maps of a galaxy,
where the unit would be in ${\rm M}_{\odot} {\rm pc}^{-2}$,
because it does not sum the mass contained in each bin,
and results depend on the resolution used.
If you wish to make such a plot anyway, you should ensure you are resolution-converged.

</div>

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

path = "osyrisdata/starformation"
data = osyris.RamsesDataset(8, path=path).load()
mesh = data["mesh"]
ind = np.argmax(mesh["density"])
center = mesh["position"][ind.values]

## Making a thick slice of gas density

To make a thick slice, we supply a `dz` argument to the `map` function.

In [ ]:
osyris.map(
    mesh.layer("density"),
    norm="log",
    dx=2000.0 * osyris.units("au"),
    dz=1500.0 * osyris.units("au"),
    origin=center,
    direction="z",
)

## Resolution

As described at the top of the notebook,
the current implementation of the thick maps means that resolution is an important parameter to consider when making thick maps.

By default, `map` will try to make the pixel size in the `z` dimension as close as possible to the pixel sizes in the horizontal and vertical directions.
This means that in the case where `dx = 2000 AU` for 256 pixels, if `dz = 1500 AU` then the number of pixels in the `z` dimension will be 192.

This can however manually be changed, using the `resolution` argument, as follows.

In [ ]:
osyris.map(
    mesh.layer("density"),
    norm="log",
    dx=2000.0 * osyris.units("au"),
    dz=1500.0 * osyris.units("au"),
    origin=center,
    direction="z",
    resolution={"z": 256},
)